In [220]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from sqlalchemy.ext.declarative import declarative_base # extract classes into tables
from sqlalchemy import Column, Integer, String, Float, MetaData 

In [221]:
file_name = ['All', 'Asian_alone', 'Black_alone', 'Hispanic_Latino', 'other_race', 'White_alone']

In [222]:
dfs=[]
for num, name in enumerate(file_name, start=0):
    csv_path ='./Resources/Los Angeles County, California_ACS_2011_2015_'+name+'_NET_RAW.csv'
    dfs.append(name+'_df_orig')
    dfs[num] = pd.read_csv(csv_path, encoding='ISO-8859-1')
    dfs[num] = dfs[num].fillna(0)
    if num == 0:
        dfs_cleaned = dfs[num]
    else:
        cols_to_use = dfs[num].columns.difference(dfs_cleaned.columns)
        dfs_cleaned = pd.merge(dfs_cleaned,(dfs[num][cols_to_use]).astype('int'), left_index=True, right_index=True, how='outer')
            
        print(cols_to_use,num)
        
    #dfs[num] = dfs[num].fillna(0)
   # print(dfs)

Index(['Asian_Alone'], dtype='object') 1
Index(['Black_or_African_American_Alone'], dtype='object') 2
Index(['Hispanic_or_Latino'], dtype='object') 3
Index(['Other_Race_Alone_or_Two_or_More_Races'], dtype='object') 4
Index(['White_Alone'], dtype='object') 5


In [223]:
dfs_cleaned = dfs_cleaned.drop(['Total'], axis=1)

In [224]:
dfs_cleaned.head(30)

,State_County_FIPS,State_FIPS,County_FIPS,County_Name,State_Name,Asian_Alone,Black_or_African_American_Alone,Hispanic_or_Latino,Other_Race_Alone_or_Two_or_More_Races,White_Alone
0,'01003','01','003',Baldwin County,Alabama,0,0,0,0,0
1,'01015','01','015',Calhoun County,Alabama,0,0,0,0,0
2,'01023','01','023',Choctaw County,Alabama,0,0,0,0,0
3,'01031','01','031',Coffee County,Alabama,0,0,0,0,0
4,'01039','01','039',Covington County,Alabama,0,0,0,0,0
5,'01041','01','041',Crenshaw County,Alabama,0,0,0,0,0
6,'01045','01','045',Dale County,Alabama,0,0,0,0,0
7,'01053','01','053',Escambia County,Alabama,0,0,0,0,0
8,'01055','01','055',Etowah County,Alabama,0,0,0,0,0
9,'01069','01','069',Houston County,Alabama,0,0,0,0,0


In [225]:
dfs_cleaned.to_csv('./Resources/Los_Angeles_County_California_ACS_2011_2015_NET_CLEANED.csv')

In [226]:
# Create SQL database for cleaned data
Base = declarative_base() 

class Station(Base):
    __tablename__= "migration"
    State_County_FIPS = Column(String(255), primary_key=True)
    State_FIPS = Column(String(255))
    County_FIPS = Column(String(255))
    County_Name = Column(String(255))
    State_Name = Column(String(255))
    White_Alone = Column(Integer)
    Asian_Alone = Column(Integer)
    Black_or_African_American_Alone = Column(Integer)
    Hispanic_or_Latino = Column(Integer)
    Other_Race_Alone_or_Two_or_More_Races = Column(Integer)
    
engine = create_engine('sqlite:///migration.sqlite')
Base.metadata.create_all(engine)
Base.metadata.tables

immutabledict({'migration': Table('migration', MetaData(bind=None), Column('State_County_FIPS', String(length=255), table=<migration>, primary_key=True, nullable=False), Column('State_FIPS', String(length=255), table=<migration>), Column('County_FIPS', String(length=255), table=<migration>), Column('County_Name', String(length=255), table=<migration>), Column('State_Name', String(length=255), table=<migration>), Column('White_Alone', Integer(), table=<migration>), Column('Asian_Alone', Integer(), table=<migration>), Column('Black_or_African_American_Alone', Integer(), table=<migration>), Column('Hispanic_or_Latino', Integer(), table=<migration>), Column('Other_Race_Alone_or_Two_or_More_Races', Integer(), table=<migration>), schema=None)})

In [227]:
# Create prepare the database for insertion 
from sqlalchemy.orm import Session
session = Session(bind=engine)

meta = MetaData()
meta.reflect(bind=engine)
migration=meta.tables['migration']

conn = engine.connect() 

In [228]:
# Insert migration data
dfs_cleaned.to_sql(name='migration', con=conn, if_exists='replace', index=False)

In [230]:
# Verify inserted data
conn = engine.connect()
stmt=migration.select().limit(5)
rs=conn.execute(stmt).fetchall()
for idx, row in enumerate(rs):
    print((rs[idx]).State_County_FIPS, (rs[idx]).County_Name, (rs[idx]).State_Name, (rs[idx]).White_Alone, (rs[idx]).Asian_Alone, 
         (rs[idx]).Black_or_African_American_Alone, (rs[idx]).Hispanic_or_Latino, (rs[idx]).Other_Race_Alone_or_Two_or_More_Races)

'01003' Baldwin County Alabama 0 0 0 0 0
'01015' Calhoun County Alabama 0 0 0 0 0
'01023' Choctaw County Alabama 0 0 0 0 0
'01031' Coffee County Alabama 0 0 0 0 0
'01039' Covington County Alabama 0 0 0 0 0
